In [2]:
class NPPArctic:
    '''
    class NPPsurf_OCNPP_comp(resultpath,savepath,mesh,npfile,first_year,last_year,
                 savefig=False, verbose=False, output=False, plotting=True, Taylor=True)
                 
    self.NPPnfesom_interp contains 2D dataset of 1x1 interpolated nanophytoplankton NPP
    self.NPPdfesom_interp contains 2D dataset of 1x1 interpolated diatom NPP
    self.NPPtfesom_interp contains 2D dataset of 1x1 interpolated of total NPP
    self.unitfesom contains str of FESOM NPP unit
    self.OCNPP contains 2D dataset of 1x1 interpolated of remotely sensed NPP
    self.lon longitude
    self.lat latitude
    '''
    
    def __init__(self,resultpath,savepath,mesh,npfile,first_year,last_year, savefig=False,
                 output=False,plotting=True, verbose = False, Taylor=True,runname = 'fesom'):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.fyear = first_year
        self.lyear = last_year
        self.savefig = savefig
        self.npfile = npfile
        self.verbose = verbose
        self.taylor = Taylor
        self.output = output
        self.plotting = plotting
        self.Taylor = Taylor
        
        from pathlib import Path
        import matplotlib.pyplot as plt
        import matplotlib.colors as colors
        import numpy as np
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import scipy.io as spio
        import cartopy.feature as cfeature
        import pyfesom2 as pf    
        from Py_f2recom_toolbox import plt_Taylor_norm

        box=[-180, 180, 60, 90]
        mapproj = pf.get_proj('np')

        if(self.verbose):
            print('Processing {0}'.format(self.resultpath))
        
        # load OCCCI CHl.a data -------------------------------------------------------------------------------------
        NPP_SAT = np.load(npfile)
        
        londic, latdic = np.meshgrid(np.arange(-179.25,180.,0.25), np.arange(-90,90.25,0.25))
        unit = 'NPP [mg C m$^{-2}$ d$^{-1}$]'

        if 'ARRIGO' in npfile:
            tag = 'LEWIS'
            print('Lewis et al. dataset selected')
            label = 'Lewis et al. [2003-2018]'
            print(' !!! Only MODIS Satellite for the 2003-2018 period !!!')
        elif 'CMEMS' in npfile:
            NPP_SAT[latdic<=60]=np.nan
            tag = 'CMEMS'
            print('Globcolour dataset selected')    
            label = 'Globcolour [2003-2018]'
            print(' !!! Only MODIS Satellite for the 2003-2018 period !!!')
        else:
            print('wrong dataset selected, please select either {Lewis} or {OCCCI}')
        
        # load FESOM mesh -------------------------------------------------------------------------------------
        years = np.arange(self.fyear, self.lyear+1,1)
        NPPn = pf.get_data(resultpath, "NPPn", years, mesh, how='mean', compute=True, silent=True)
        NPPd = pf.get_data(resultpath, "NPPd", years, mesh, how='mean', compute=True, silent=True)
        
        cocco_path = Path(self.resultpath + '/NPPc.fesom.'+str(years[0])+'.nc') # assuming that coccos were used for the entire simulation if they were used in the first year of simulation
        phaeo_path = Path(self.resultpath + '/NPPp.fesom.'+str(years[0])+'.nc') # assuming that phaeo was used for the entire simulation if they were used in the first year of simulation
        
        if cocco_path.is_file():
            NPPc = pf.get_data(self.resultpath, "NPPc", years, mesh, how="mean", compute=True, silent=True)
            
            if phaeo_path.is_file():
                NPPp = pf.get_data(self.resultpath, "NPPp", years, mesh, how="mean", compute=True, silent=True)
                
                print('4-phytoplankton model is used')
                NPP_MODEL = 365* (NPPd+NPPn+NPPc+NPPp)*12.01 /1e3
            else: 
                print('3-phytoplankton model is used')
                NPP_MODEL = 365* (NPPd+NPPn+NPPc)*12.01 /1e3 
            
        else:
            print('2-phytoplankton model is used')
            NPP_MODEL = 365* (NPPd+NPPn)*12.01 /1e3

        # interpolate FESOM CHl.a to regular -------------------------------------------------------------------------------------
        NPPfesom_interp = pf.fesom2regular(
                data = NPP_MODEL,
                mesh = mesh,
                lons = londic, 
                lats = latdic)

        NPPfesom_interp_log10 = np.log10(NPPfesom_interp)
        fesom_label = 'FESOM-REcoM NPP {0}-{1}'.format(self.fyear,self.lyear)        

        # apply sea mask to OCCCI as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        NPPsat_ma = np.copy(NPP_SAT)
        NPPsat_ma[~np.isfinite(NPPfesom_interp)] = np.nan
        NPPsat_ma_log10 = np.log10(NPPsat_ma)

        # plotting -------------------------------------------------------------------------------------
        levels = 100*np.array([0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,
                                   0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,
                                   1,2,3,4,5])
        ticks = [0,1,3,5,7,10,30,50,70,100,300,500]
        ticks_label = ['0','1','3','5','7','10','30','50','70','100','300','500']

        def mygrid(m):
            m.add_feature(cfeature.LAND, zorder=1, edgecolor='none', facecolor='gray')
                
        fig, axes = plt.subplots(1,3, 
                                     subplot_kw=dict(projection=mapproj),
                                     gridspec_kw={'hspace': 0.01, 'wspace': 0.1},
                                     figsize=(20,7), constrained_layout=False)     

        # REcoM
        m1 = axes[0]
        f1 = m1.pcolormesh(londic, latdic, NPPfesom_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                                   #vmin=1e-3,vmax=5e3)
        mygrid(m1)
        m1.set_title(fesom_label, fontsize=16)


        # Satellite
        m2 = axes[1]
        f2 = m2.pcolormesh(londic, latdic, NPPsat_ma, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
        mygrid(m2)
        m2.set_title(label, fontsize=16)

        # add one colorbar for first row plots below figure
        cbar = fig.colorbar(f1,
                                ax = axes[:2], 
                                location ='bottom',
                                extend = 'max',
                                ticks = ticks,
                                fraction=0.046, pad=0.04) 
        #cbar.ax.tick_params(labelsize=14)
        cbar.ax.set_xticklabels(ticks_label, fontsize=16) 
        cbar.set_label(unit, fontsize=16)
        cbar.ax.set_xticklabels(cbar.ax.get_xticklabels(), rotation=45)
                
        # REcoM - Satellite
        levels_diff = 100*np.arange(-3,3,0.125)
        m3 = axes[2]
        f3 = m3.pcolormesh(londic, latdic, NPPfesom_interp - NPPsat_ma, 
                                   transform = ccrs.PlateCarree(),cmap = 'RdBu_r',
                                   norm=colors.BoundaryNorm(boundaries=levels_diff, ncolors=256))
        mygrid(m3)
        m3.set_title('FESOM-REcoM - '+label, fontsize=16)

        # add one colorbar for difference plot below figure
        cbar = fig.colorbar(f3,
                            ax = axes[2], 
                            orientation = 'horizontal',
                            #location ='bottom',
                            ticks = [-300,-200,-100,0,100,200,300],
                            extend = 'both',
                            fraction=0.046, pad=0.04) 
        cbar.ax.tick_params(labelsize=14)
        cbar.set_label(unit, fontsize=16)
        cbar.ax.set_xticklabels(cbar.ax.get_xticklabels(), rotation=45)
                
        m1.text(-0.12, 1.05, 'A', transform=m1.transAxes,
                        size=30, weight='bold')
        m2.text(-0.12, 1.05, 'B', transform=m2.transAxes,
                            size=30, weight='bold')
        m3.text(-0.12, 1.05, 'C', transform=m3.transAxes,
                            size=30, weight='bold')
                    
        m1.set_extent(box, ccrs.PlateCarree())
        m2.set_extent(box, ccrs.PlateCarree())
        m3.set_extent(box, ccrs.PlateCarree())

        # fig export  -------------------------------------------------------------------------------------
        if(self.savefig==True):
                plt.savefig(self.savepath+self.runname+'_'+'ArcNPP_'+tag+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
                plt.savefig(self.savepath+self.runname+'_'+'ArcNPP_'+tag+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                        bbox_inches='tight')
        plt.show(block=False)

        if(self.Taylor):
            # statistics  -------------------------------------------------------------------------------------            
            # preparation of datasets
            if np.isnan(np.min(NPPsat_ma_log10)): print('WARNING: Satellite field contains NaNs')
            if np.isnan(np.min(NPPfesom_interp_log10)): print('WARNING: FESOM field contains NaNs')

            # get statistics only from valid OCCCI gridpoints 
            ind_stat = np.where(np.isfinite(NPPsat_ma_log10))

            title = 'log10 NPP'
            print('\nStatistics for '+title)
            plt_Taylor_norm(NPPsat_ma_log10[ind_stat],NPPfesom_interp_log10[ind_stat],
                                    mask=True,title=title)

            
            #plt.show(block=False) 
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):
                plt.savefig(self.savepath+self.runname+'_'+'ArcNPP_'+tag+'_'+str(years[0])+'to'+str(years[-1])+'_Taylor.png', 
                        dpi = 300, bbox_inches='tight')
                plt.savefig(self.savepath+self.runname+'_'+'ArcNPP_'+tag+'_'+str(years[0])+'to'+str(years[-1])+'_Taylor.pdf', 
                        bbox_inches='tight')
            plt.show(block=False)  
        
        if(self.output):
            self.lon = londic
            self.lat = latdic
            self.chl_oc = NPPsat_ma
            self.chl_fesom = NPPfesom_interp
            self.unit = unit